----
<img src="../Example.DataLibrary.Python-main/files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## RDP Middle Office News Use Case
Request company PermID, OrgName and statusPass for a large list of CUSIP, ISIN or SEDOLs:

Symbol conversion via RDP Search - **RDP Search permissions are required**

1. Read SEDOLs from file and chunk the list of SEDOLs
2. Map SEDOLs to PermIds
3. Request OrgName and Status

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Example.DataLibrary.Python-main/Configuration"

### Imports and Setup

In [2]:
import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion
import pandas as pd

In [3]:
#  Debug only
#import logging
#logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)

In [4]:
pd.set_option('display.max_rows', None)

### Read file with IDs

In [5]:
defaultIDs =  ['BDZDRC5','BFZP189','BGY7Q00']
def readIDFile(filename):
    if filename != '':
        f = open(filename, "r")
        idList = f.read().splitlines()
        print('Read '+str(len(idList))+' IDs from file '+filename)
    else:
        idList = defaultIDss
    return idList
myIDs = readIDFile('SEDOLs.csv')

Read 2754 IDs from file SEDOLs.csv


## Open RDP Session
*** The session selected is required to have permissions trapi.data.get.data.read and trapi.data.serach.read
or trapi.data.searclite.read, contact your Refinitiv account manager to verify the permissions in place

In [6]:
rd.open_session('desktop.workspace')
#rd.open_session(name='desktop.workspace',config_name='../../../Configuration/refinitiv-data.config.json')

<refinitiv.data.session.Definition object at 0x2196bbe2e20 {name='workspace'}>

## Retrieve data

### Conversion of multiple symbols to multiple types 

In [7]:
response = symbol_conversion.Definition(symbols=defaultIDs).get_data()

response.data.df

,DocumentTitle,RIC,IssueISIN,CUSIP,SEDOL,TickerSymbol,IssuerOAPermID
BDZDRC5,"Premier Inc, Ordinary Share, Class A, NASDAQ S...",PINC.OQ,US74051N1028,74051N102,BDZDRC5,PINC,5040244748
BFZP189,"Gritstone bio Inc, Ordinary Share, Frankfurt S...",2JQ.F,US39868T1051,39868T105,BFZP189,2JQ,5046716076
BGY7Q00,"Yihai International Holding Ltd, Ordinary Shar...",1579.HS,KYG984191075,<NA>,BGY7Q00,1579,5050707044


### Conversion of mulitple symbols to RICs and OA_PERM_IDs

In [8]:
# Symbol types:
# - symbol_conversion.RIC => RIC
# - symbol_conversion.ISIN => IssueISIN
# - symbol_conversion.CUSIP => CUSIP
# - symbol_conversion.SEDOL => SEDOL
# - symbol_conversion.TICKER_SYMBOL => TickerSymbol
# - symbol_conversion.OA_PERM_ID => IssuerOAPermID
# - symbol_conversion.LIPPER_ID => FundClassLipperID

response = symbol_conversion.Definition(
    symbols=defaultIDs,
    from_symbol_type=symbol_conversion.SymbolTypes.SEDOL,
    to_symbol_types=[
        symbol_conversion.SymbolTypes.RIC,
        symbol_conversion.SymbolTypes.OA_PERM_ID
    ],
).get_data()

response.data.df

,RIC,IssuerOAPermID
BDZDRC5,PINC.OQ,5040244748
BFZP189,2JQ.F,5046716076
BGY7Q00,1579.HS,5050707044


In [9]:
rd.get_data(
    universe=defaultIDs,
    fields=['TR.CommonName','TR.OrganizationStatusCode','TR.Revenue']    
)

,Instrument,Company Common Name,Organization Status Code,Revenue
0,BDZDRC5,Premier Inc,Act,1721152000
1,BFZP189,Gritstone bio Inc,Act,48214000
2,BGY7Q00,Yihai International Holding Ltd,Act,5942617000


### Convert 2K list

In [10]:
response = symbol_conversion.Definition(
    symbols=myIDs,
    from_symbol_type=symbol_conversion.SymbolTypes.SEDOL,
    to_symbol_types=[
        symbol_conversion.SymbolTypes.RIC,
        symbol_conversion.SymbolTypes.OA_PERM_ID
    ],
).get_data()

response.data.df

,RIC,IssuerOAPermID
BFZP189,2JQ.F,5046716076
BGY7Q00,1579.HS,5050707044
2019479,BXP.N,4295903585
BK81PL0,307.BN,5001433891
B038282,BFIN.OQ,4295899581
5964899,LNC.BE,4295904418
B1LJ7G7,REED.OQ,4295901921
2418968,SW.TO,4295862380
BK9RXB3,GLG.OQ,5040042275
BK947J0,GTU2.F,4295908396


### Chunk Long List- Define a Helper Function 

In [12]:
def list_to_chunks(long_list, chunk_size):
    chunked_list = list()
    for i in range(0, len(long_list), chunk_size):
        chunked_list.append(long_list[i:i+chunk_size])
    return chunked_list
#CHUNK_SIZE = 500
#myIDChunks = list_to_chunks(myIDs, CHUNK_SIZE)
#print('My chunks are ',myIDChunks)
#print('Number of chunks is: ',len(myIDChunks))

###  Request CommonName and OrganizationStatusCode - Loop Permid Chunks
The request is partitioned into chunks of size N (default is 1000), as requesting the full list of a larger size may result in an error

In [13]:
N = 1000
permids =response.data.df['RIC'].astype(str).values.tolist()
myPermidChunks = list_to_chunks(permids, N)
dataAll = []
dfAll = pd.DataFrame()
for i in range(len(myPermidChunks)):
    print('Iteration #',i)
    dfResp = rd.get_data(
        universe=myPermidChunks[i],
        fields=['TR.CommonName','TR.OrganizationStatusCode','TR.Revenue']    
    )
    dataAll.append(dfResp)
    #print(resp)
dfAll = pd.concat(dataAll,ignore_index=True)
print('Displaying full result:')
display(dfAll)

Iteration # 0
Iteration # 1
Iteration # 2
Displaying full result:


,Instrument,Company Common Name,Organization Status Code,Revenue
0,2JQ.F,Gritstone bio Inc,Act,48214000
1,1579.HS,Yihai International Holding Ltd,Act,5942617000
2,BXP.N,Boston Properties Inc,Act,2888621000
3,307.BN,Shopify Inc,Act,4611856000
4,BFIN.OQ,BankFinancial Corp,Act,
5,LNC.BE,Lincoln National Corp,Act,
6,REED.OQ,Reed's Inc,Act,41615000
7,SW.TO,Sierra Wireless Inc,Act,473209000
8,GLG.OQ,TD Holdings Inc,Act,
9,GTU2.F,Oncternal Therapeutics Inc,Act,4315000


## Close the session

In [36]:
rd.close_session()